In [ ]:
#include <string>
#include <filesystem>
#include <fstream>
#include <chrono>

class PerformanceAnalysis
{
public:
    std::string path = "../merged/";

    double RuntimeMeasure(std::string path, int nFiles, int nThreads)
    {
        if (nThreads != 0) 
        {
            ROOT::EnableImplicitMT(nThreads);
        }

        std::vector<std::filesystem::path> filesToRead = {};

        for (auto const& file : std::filesystem::directory_iterator(path))
        {
            filesToRead.push_back(file.path());
        }

        std::chrono::steady_clock::time_point startTime = std::chrono::steady_clock::now();

        ROOT::RDataFrame df("rootuple/CandidateTree", "../merged/");

        auto cut = df.Filter("candidate_charge == 0")
                     .Filter("candidate_cosAlpha > 0.99")
                     .Filter("candidate_vProb > 0.05")
                     .Filter("candidate_lxy / candidate_lxyErr > 3.0")
                     .Filter("ditrack_mass > 1.014")
                     .Filter("ditrack_mass < 1.024")
                     .Filter("candidate_vMass > 5.33")
                     .Filter("candidate_vMass < 5.40");

        cut.Display()->Print();

        return (std::chrono::steady_clock::now() - startTime).count() / 1000000000.0;
    }

    void RuntimeVsThreads(std::string path, int nFiles, int maxThreads, int step, int nLoops, std::string target_dir)
    {
        std::string resultPath = "runtime_tests_rdf/" + target_dir + "/runtime_vs_threads_" + std::to_string(nFiles) + "_" + std::to_string(maxThreads) + "_" + std::to_string(step) + "_" + std::to_string(nLoops) + ".csv";
        if (!std::filesystem::exists("runtime_tests_rdf/" + target_dir))
        {
            std::filesystem::create_directory("runtime_tests_rdf/" + target_dir);
        }

        std::ofstream file;
        if (!std::filesystem::exists(resultPath))
        {
            file.open(resultPath);
            for (int i = 0; i < maxThreads; i+=step)
            {
                file << i << ",";
            }
            file << "\n";
        }

        for (int i = 0; i < nLoops; i++)
        {
            for (int j = 0; j < maxThreads; j+=step)
            {
                file << RuntimeMeasure(path, nFiles, j) << ",";
            }
            file << "\n";
        }   
        file.close();
    }
};